In [1]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
from scipy.optimize import least_squares, curve_fit
from matplotlib import pylab as plt
import pandas as pd
import glob
import time
from tqdm.notebook import tqdm
import os
import random

import warnings
from scipy.optimize import OptimizeWarning
warnings.simplefilter("error", OptimizeWarning)

# import seaborn as sns
# sns.set()

In [2]:
def plot_allbands_df(df,title=None,ax=plt):
    ugrizYcolors = ["violet","green","red","darkred","grey","black"]
    for i,band in enumerate("ugrizY"):
        subdf = df[df["band"]==band].reset_index(drop=True)
        x = subdf["mjd"]
        y = subdf["flux"]
        yerr = subdf["fluxerr"]
#         ax.errorbar(x,y,yerr=yerr,ls = "None",color=ugrizYcolors[i])
        ax.scatter(x,y,s=5,label=band,color=ugrizYcolors[i])
        if title is not None:
            if ax is not plt:
                ax.set_title(title)
            else:
                ax.title(title)
    return ax

In [3]:
filels = glob.glob("csv_data/*.csv")

# NEW

In [206]:
file = random.sample(filels,1)[0]
title=file.split("/")[-1]
title_class = title.split("_")[-1].split(".")[0]
print(title)

241704_SNIa-91bg.csv


In [219]:
def center_bigdf(df):
    
    df["mjd"] = df["mjd"] - df["mjd"][0]
    df["ratio"] = abs(df["flux"]/df["fluxerr"])

    df["ratio2bool"] = 0

    subdf_detected = df[df["detected_bool"]==1]
    subdf_ratio2 = df[df["ratio"]>=2]


    smalldf = subdf_ratio2.loc[abs(subdf_ratio2.mjd.diff(periods=-1))<=5]
    for i in range(3):
        smalldf = smalldf[abs(smalldf.mjd.diff(periods=-1))<=5]

    if len(subdf_detected)>0 and len(smalldf)>0:
        detected_bool_minmjd = subdf_detected.mjd.values[0]
        custom_bool_minmjd = smalldf.mjd.values[0]

        diffmjd = abs(detected_bool_minmjd - custom_bool_minmjd)/2

        minmjdstart = min(detected_bool_minmjd, custom_bool_minmjd)
        maxmjdstart = max(detected_bool_minmjd, custom_bool_minmjd)

        mjdlim_lower = minmjdstart - (100-diffmjd)
        mjdlim_upper = maxmjdstart + (100-diffmjd)

    elif len(subdf_detected)>0:
        detected_bool_minmjd = subdf_detected.mjd.values[0]
        mjdlim_lower = detected_bool_minmjd - 100
        mjdlim_upper = detected_bool_minmjd + 100

    elif len(smalldf)>0:
        custom_bool_minmjd = smalldf.mjd.values[0]
        mjdlim_lower = custom_bool_minmjd - 100
        mjdlim_upper = custom_bool_minmjd + 100

    else:
        mjdlim_lower = 0
        mjdlim_upper = 200

    assert (mjdlim_upper-mjdlim_lower)==200

    df.loc[(df["mjd"] >= mjdlim_lower) & (df["mjd"] <= mjdlim_upper),"ratio2bool"] = 1
    
    newdf = df[df["ratio2bool"]==1].reset_index(drop=True)
    
    return newdf

In [221]:
366.9827 - 495.7503

-128.76759999999996

In [222]:
center_bigdf(df)

,mjd,band,flux,fluxerr,detected_bool,ratio,ratio2bool
0,366.9827,r,-0.309199,1.199211,0,0.257835,1
1,366.9904,g,0.583735,1.354673,0,0.430905,1
2,366.9981,i,-1.484780,1.522391,0,0.975295,1
3,367.0090,z,-1.214377,2.026973,0,0.599109,1
4,367.0200,Y,5.033326,4.792842,0,1.050176,1
...,...,...,...,...,...,...,...
95,491.7556,u,-2.038524,1.925449,0,1.058727,1
96,492.7484,u,1.656206,1.680197,0,0.985721,1
97,493.7494,u,-1.944438,1.867722,0,1.041075,1
98,494.7494,u,0.646264,1.574601,0,0.410430,1


In [214]:
df = pd.read_csv(file)

df["mjd"] = df["mjd"] - df["mjd"][0]
df["ratio"] = abs(df["flux"]/df["fluxerr"])

df["ratio2bool"] = 0

subdf_detected = df[df["detected_bool"]==1]
subdf_ratio2 = df[df["ratio"]>=2]


smalldf = subdf_ratio2.loc[abs(subdf_ratio2.mjd.diff(periods=-1))<=5]
for i in range(3):
    smalldf = smalldf[abs(smalldf.mjd.diff(periods=-1))<=5]

if len(subdf_detected)>0 and len(smalldf)>0:
    detected_bool_minmjd = subdf_detected.mjd.values[0]
    custom_bool_minmjd = smalldf.mjd.values[0]
    
    diffmjd = abs(detected_bool_minmjd - custom_bool_minmjd)/2
    
    minmjdstart = min(detected_bool_minmjd, custom_bool_minmjd)
    maxmjdstart = max(detected_bool_minmjd, custom_bool_minmjd)
    
    mjdlim_lower = minmjdstart - (100-diffmjd)
    mjdlim_upper = maxmjdstart + (100-diffmjd)
    
elif len(subdf_detected)>0:
    detected_bool_minmjd = subdf_detected.mjd.values[0]
    mjdlim_lower = detected_bool_minmjd - 100
    mjdlim_upper = detected_bool_minmjd + 100
    
elif len(smalldf)>0:
    custom_bool_minmjd = smalldf.mjd.values[0]
    mjdlim_lower = custom_bool_minmjd - 100
    mjdlim_upper = custom_bool_minmjd + 100

else:
    mjdlim_lower = 0
    mjdlim_upper = 200
    
assert (mjdlim_upper-mjdlim_lower)==200

df.loc[(df["mjd"] >= mjdlim_lower) & (df["mjd"] <= mjdlim_upper),"ratio2bool"] = 1

In [215]:
df

,mjd,band,flux,fluxerr,detected_bool,ratio,ratio2bool
0,0.0000,r,0.626059,1.362348,0,0.459544,0
1,0.0076,g,-0.475598,1.154428,0,0.411977,0
2,0.0152,i,-1.771064,1.768351,0,1.001534,0
3,0.0261,z,-1.613671,2.608133,0,0.618707,0
4,0.0371,Y,-3.711906,5.376524,0,0.690391,0
...,...,...,...,...,...,...,...
325,853.8084,r,0.473870,0.941466,0,0.503332,0
326,853.8160,g,-0.778719,1.122279,0,0.693873,0
327,853.8236,i,1.448867,1.380313,0,1.049666,0
328,853.8345,z,1.403155,2.044781,0,0.686213,0
